# Exercise 5

In [586]:
import numpy as np
import scipy.stats as stats
import math
import statistics as st

print("\nIntegral's exact solution:",np.exp(1)-1)


Integral's exact solution: 1.718281828459045


1. Estimate the integral of the exponential function from 0 to 1 by simulation (the crude Monte Carlo estimator). Use eg. an estimator based on 100 samples and present the result as the point estimator and a confidence interval.

In [526]:
def cmc(n):
    x = []
    for i in range(n):
        u = np.random.uniform(0,1)
        x.append(np.exp(u))
    res = sum(x)/n
    var = np.var(x)
    return res, var

res, var = cmc(100)
lb = res - stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
ub = res + stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
print(res,var,lb,ub)

1.7983227765189471 0.2945747418315329 1.7082139898841489 1.8884315631537454


2. Estimate the integral of the exponential function from 0 to 1 using antithetic variables, with comparable computer ressources.

In [527]:
def av(n):
    x= []
    for i in range(n):
        u = np.random.uniform(0,1)
        y = (np.exp(u)+np.exp(1-u))/2
        x.append(y)
    res = sum(x)/n
    var = np.var(x)
    return res, var

res, var = av(100)
lb = res - stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
ub = res + stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
print(res,var,lb,ub)

1.7268817909405243 0.004100966096902039 1.716249851888876 1.7375137299921726


3. Estimate the integral of the exponential function from 0 to 1 using a control variable, with comparable computer ressources.

In [528]:
def cv(n):
    z = []
    u = np.random.uniform(0,1,n)
    c = -np.cov(u,np.exp(u))[0, 1]/np.var(u)
    for i in range(n):
        z.append(np.exp(u[i])+c*(u[i]-0.5))
    res = sum(z)/n
    var = np.var(z)
    return res, var

res, var = cv(100)
lb = res - stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
ub = res + stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
print(res,var,lb,ub)

1.716161951209148 0.003431013180648312 1.706437153366154 1.725886749052142


4. Estimate the integral of the exponential function from 0 to 1 using stratified sampling, with comparable computer ressources.

In [594]:
def strat(n, m):
    w = []
    for i in range(n):
        wi = 0
        for j in range(m):
            u = np.random.uniform(j/m, (j + 1)/m)
            wi += np.exp(j/m+u/m)
        w.append(wi/m)
    res = sum(w)/n
    var = np.var(w)
    return res, var

res, var = strat(100, 10)
lb = res - stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
ub = res + stats.t.ppf(0.95, 100) * (math.sqrt(var)/math.sqrt(100))
print(res,var,lb,ub)

1.732224633253939 2.837749652480999e-06 1.7319449564626046 1.7325043100452735


5. Use control variates to reduce the variance of the estimator in exercise 4 (Poisson arrivals).

In [530]:
su = 10; mst = 8; mct = 1; c = 10000; n = 10

def bc1(su, mst, mct, c):
    res = 0
    t = np.cumsum(np.random.exponential(mct, c))
    st = [0] * su
    for i in range(c):
        if t[i] > min(st):
            st[st.index(min(st))] = np.random.exponential(mst) + t[i]
        else:
            res +=1
    return res/c, np.mean(t)

res = [bc1(su, mst, mct, c) for i in range(n)]
x,y = zip(*res)

def cv2(x,y):
    c = -np.cov(x,y)[0, 1]/np.var(y)
    res = x+c*(y-np.mean(y))
    var = np.var(res)
    return res, var

res, var = cv2(x, y)
print(var)

1.9610225520991113e-05


6. Demonstrate the effect of using common random numbers in exercise 4 for the difference between Poisson arrivals (Part 1) and a renewal process with hyperexponential interarrival times.

7. For a standard normal random variable Z ∼ N(0, 1) using the crude Monte Carlo estimator estimate the probability Z > a. Then try importance sampling with a normal density with mean a and
variance σ2. For the expirements start using σ2 = 1, use different values of a (e.g. 2 and 4), and different sample sizes. If time permits experiment with other values for σ2. Finally discuss the
efficiency of the methods.

In [615]:
n = 10000; a = 1; sigma = 1
exp_prob = 1-stats.norm.cdf(a, 0, 1)

def cmc1(n,a):
    u = np.random.uniform(size=n)
    N = stats.norm.ppf(u)
    res = sum(N>a)/n
    return res

res1 = cmc1(n,a)

def cmc2(n,a):
    u = np.random.uniform(size=n)
    z = 1/(np.sqrt(2*np.pi))*(a)/(u**2)*np.exp(-0.5*(a/u)**2)
    res = sum(z)/n
    return res

res2 = cmc2(n,a)

def imps1(n, a, sigma):
    y = np.random.normal(a, sigma, n)
    h = y>a
    f = stats.norm.pdf(y)
    g = stats.norm.pdf(y, a, sigma)
    res = sum(h*f/g)/n
    return res

res3 = imps1(n, a, sigma)

print("\n Expected Probability", exp_prob)
print("\n Crude Monte Carlo:", res1, res2)
print("\n Importance Sampling:", res3)


 Expected Probability 0.15865525393145707

 crude Monte Carlo: 0.1632 0.15839898632708352

 Importance Sampling: 0.1596074398939642


8. Use importance sampling with g(x) = λ exp (−λ ∗ x) to calculate the integral  of Question 1. Try to find the optimal value of λ by calculating the variance of h(X)f(X)/g(X) and verify by simulation.

In [577]:
def imps2(l, n):
    u = np.random.uniform(size=n)
    y = l*np.exp(-l*u)
    #y = stats.expon.ppf(u, scale=1/l)
    h = np.exp(y)
    f = y * (y <= 1)
    g = l*np.exp(-l*y)
    #g = stats.expon.pdf(y, scale=1/l)
    res = sum(h*f/g)/n
    var = np.var(h*f/g)
    return res, var

n=10000
ls = np.linspace(1, 10, 10000)
varl = []
resl = []
error = 300
optres = 0
optvar = 0
for l in ls:
    res, var = imps2(l, n)
    resl.append(res)
    varl.append(var)
    new_error = np.abs(res-(np.exp(1)-1))
    if new_error<error:
        error = new_error
        optres = res
        optvar = var
        optl1 = l

print("\nLambda for which IS estimator approaches the exact solution:", optl1)
print("Corresponding integral value and variance:",optres,optvar)
print("Simulated values:",imps2(optl1,n))

optl2 = ls[np.argmin(varl)]
print("\nLambda that minimises variance:", optl2)
print("Corresponding integral value and minimum variance:",resl[np.argmin(varl)],min(varl))
print("Simulated values:",imps2(optl2,n))


Lambda for which IS estimator approaches the exact solution: 3.8037803780378034
Corresponding integral value and variance: 1.7182297430783398 21.719947569201505
Simulated values: (1.6887181755409053, 21.32714176134239)

Lambda that minimises variance: 1.0054005400540054
Corresponding integral value and minimum variance: 2.576925082822111 2.990418031211429
Simulated values: (2.6435157874081994, 3.1454635908486925)


9. For the Pareto case derive the IS estimator for the mean using the first moment distribution as sampling distribution. Is the approach meaningful? and could this be done in general? With this insight could you change the choice of g(x) in the previous question (Question 8) such that importance sampling would reduce the variance? You do not need to implement this, as long as you can argue, what should happen.

In [690]:
def imps3(n,b,k):
    u = np.random.uniform(size=n)
    y = stats.pareto.ppf(u, b, loc=0, scale=k)
    h = np.mean(y)
    f = stats.pareto.pdf(y, b, loc=0, scale=k) 
    g = k*b/(k-1)
    return np.sum(h*f/g)/n
imps3(10000,1,5)

2.9846659970952385